# SNN神经网络权重部署

In [9]:
import numpy as np

3bit（8级别）差分矩阵元素对应表

8级别的衰减值分别为：-1dB，-1.66dB，-2.44dB，-3.39dB，-4.61dB，-6.30dB，-9.14dB，-20dB

为了确保传输率的增量是线性的，dB值就不能是线性的

| 权重 | 差分正极dB    | 对应传输率 | 差分负极dB     | 对应传输率  |
|------|--------------|-----------|---------------|-------------|
|7     | -1.00dB      |   0.794   |    -20.0dB     |    0.010   |
|6     | -1.00dB      |   0.794   |    -9.14dB     |    0.122   |
|5     | -1.00dB      |   0.794   |    -6.30dB     |    0.234   |
|4     | -1.00dB      |   0.794   |    -4.61dB     |    0.346   |
|3     | -1.00dB      |   0.794   |    -3.39dB     |    0.458   |
|2     | -1.00dB      |   0.794   |    -2.44dB     |    0.570   |
|1     | -1.00dB      |   0.794   |    -1.66dB     |    0.682   |
|0     | -1.00dB      |   0.794   |    -1dB        |    0.794   |
|-1    | -1.66dB      |   0.682   |    -1dB        |    0.794   |
|-2    | -2.44dB      |   0.570   |    -1dB        |    0.794   |
|-3    | -3.39dB      |   0.458   |    -1dB        |    0.794   |
|-4    | -4.61dB      |   0.346   |    -1dB        |    0.794   |
|-5    | -6.30dB      |   0.234   |    -1dB        |    0.794   |
|-6    | -9.14dB      |   0.122   |    -1dB        |    0.794   |
|-7    | -20.0dB      |   0.010   |    -1dB        |    0.794   |

In [10]:
# 正极权重映射dB对应表
mapping_dict_pos = {7: -1, 6: -1, 5: -1, 4: -1, 3:-1, 2:-1, 1:-1, 0:-1, 
                    -1:-1.66, -2:-2.44, -3:-3.39, -4:-4.61, -5:-6.30, -6:-9.14, -7:-20, -8:-20}
# 负极权重映射dB对应表
mapping_dict_neg = {7: -20, 6: -9.14, 5: -6.30, 4: -4.61, 3:-3.39, 2:-2.44, 1:-1.66, 0:-1,
                    -1:-1, -2:-1, -3:-1, -4:-1, -5:-1, -6:-1, -7:-1, -8:-1}

def weight_to_dB(weight, mapping_dict_pos, mapping_dict_neg):
    """
    将输入的numpy数组按照给定的映射表转换为新的数组。

    参数:
        weight (numpy.array): 输入的numpy数组。
        mapping_dict (dict): 映射表，键是输入数组中的值，值是映射后的值。

    返回:
        W_p_dB: 映射后的正极dB数组。
        W_n_dB: 映射后的负极dB数组。
    """
    # 创建2个与输入数组形状相同的空数组，用于存储映射后的值
    W_p_dB = np.zeros_like(weight)
    W_n_dB = np.zeros_like(weight)

    # 遍历输入数组的每个元素，并根据正极映射表更新输出数组
    for key, value in mapping_dict_pos.items():
        W_p_dB[weight == key] = value

    # 遍历输入数组的每个元素，并根据负极映射表更新输出数组
    for key, value in mapping_dict_neg.items():
        W_n_dB[weight == key] = value

    return W_p_dB, W_n_dB

## SNN第1层输入脉冲信号

In [11]:
# 输入向量（每一行代表一个时间步长，共30个时间步长）
# 鸢尾花数据集30个测试集数据
index_0_layer_1 = np.array(
             [[1e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 1e-3, 1e-3],
              [1e-3, 1e-3, 1e-3, 0e-3],
              [1e-3, 1e-3, 1e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 1e-3, 0e-3],
              [0e-3, 1e-3, 1e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 1e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [0e-3, 1e-3, 1e-3, 0e-3],
              [0e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 1e-3],
              [0e-3, 1e-3, 1e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3],
              [1e-3, 0e-3, 0e-3, 0e-3],
              [1e-3, 1e-3, 0e-3, 0e-3]]
              )

index_10_layer_1 = np.array(
             [[1e-3, 1e-3, 1e-3, 0e-3], # 第0个时间步长
              [1e-3, 0e-3, 1e-3, 0e-3], # 第1个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第2个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第3个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第4个时间步长
              [0e-3, 1e-3, 1e-3, 0e-3], # 第5个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第6个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第7个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第8个时间步长
              [1e-3, 1e-3, 0e-3, 0e-3], # 第9个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第10个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第11个时间步长
              [0e-3, 1e-3, 1e-3, 1e-3], # 第12个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第13个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第14个时间步长
              [0e-3, 1e-3, 1e-3, 0e-3], # 第15个时间步长
              [1e-3, 0e-3, 1e-3, 0e-3], # 第16个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第17个时间步长
              [1e-3, 0e-3, 1e-3, 0e-3], # 第18个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第19个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第20个时间步长
              [1e-3, 1e-3, 0e-3, 0e-3], # 第21个时间步长
              [0e-3, 1e-3, 1e-3, 0e-3], # 第22个时间步长
              [1e-3, 0e-3, 0e-3, 0e-3], # 第23个时间步长
              [1e-3, 0e-3, 1e-3, 1e-3], # 第24个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第25个时间步长
              [1e-3, 0e-3, 1e-3, 0e-3], # 第26个时间步长
              [1e-3, 0e-3, 1e-3, 1e-3], # 第27个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第28个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3]] # 第29个时间步长
              )

index_20_layer_1 = np.array(
             [[1e-3, 0e-3, 1e-3, 1e-3], # 第0个时间步长
              [1e-3, 0e-3, 1e-3, 1e-3], # 第1个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第2个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第3个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第4个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第5个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第6个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第7个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第8个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第9个时间步长
              [1e-3, 1e-3, 0e-3, 1e-3], # 第10个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第11个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第12个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第13个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第14个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第15个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第16个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第17个时间步长
              [1e-3, 0e-3, 1e-3, 1e-3], # 第18个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第19个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第20个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第21个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第22个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第23个时间步长
              [1e-3, 0e-3, 0e-3, 1e-3], # 第24个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第25个时间步长
              [1e-3, 1e-3, 0e-3, 1e-3], # 第26个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第27个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第28个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3]] # 第29个时间步长
              )

## 神经网络第1层量化后权重部署

In [18]:
print("-----神经网络第1层量化后权重部署-----")

# 输入向量（从上一个代码块中选择）
# x = index_10_layer_1
# print("输入向量功率 = \n", x)

# 测试输入向量
x = np.array([6.7, 3, 5.2, 2.3])
print("输入向量功率 = \n", x)

# 原始权重矩阵数据
W = np.array([[ 2.,  3., -3., -8.],
              [ 2.,  3., -3., -3.],
              [ 1., -0.,  2., -3.],
              [ 2.,  0.,  1.,  2.]])
print("原始权重矩阵数据 = \n", W)

# 将pytorch权重矩阵转置
W = W.T
print("将pytorch权重矩阵转置 = \n", W)

# 原始矩阵乘法输出值
y = np.matmul(x, W)
print("原始矩阵乘法输出结果 = ", y)

# 将权重映射到dB
W_p_dB, W_n_dB = weight_to_dB(W, mapping_dict_pos, mapping_dict_neg)

# 差分正极dB矩阵
# W_p_dB = np.array([[-1, -1, -3.39, -20],
#                    [-1, -1, -3.39, -3.39],
#                    [-1, -1, -1, -3.39],
#                    [-1, -1, -1, -1]])
print("差分正极dB矩阵 = \n", W_p_dB)

# 差分负极dB矩阵
# W_n_dB = np.array([[-2.44, -3.39, -1, -1],
#                    [-2.44, -3.39, -1, -1],
#                    [-1.66, -1, -2.44, -1],
#                    [-2.44, -1, -1.66, -2.44]])
print("差分负极dB矩阵 = \n", W_n_dB)

# 差分正极传输率矩阵
# W_p = np.array([[0.794, 0.794, 0.199, 0.01],
#                 [0.794, 0.794, 0.199, 0.199],
#                 [0.794, 0.794, 0.794, 0.199],
#                 [0.794, 0.794, 0.794, 0.794]])

# 差分负极传输率矩阵
# W_n = np.array([[0.316, 0.199, 0.794, 0.794],
#                 [0.316, 0.199, 0.794, 0.794],
#                 [0.501, 0.794, 0.316, 0.794],
#                 [0.316, 0.794, 0.501, 0.316]])

# 传输率转化为dB
# W_p_dB = 10 * np.log10(W_p / 1)
# W_n_dB = 10 * np.log10(W_n / 1)
# print("差分正极矩阵传输率转化为dB = \n", W_p_dB)
# print("差分负极矩阵传输率转化为dB = \n", W_n_dB)

# dB转化为传输率
W_p = np.power(10, W_p_dB / 10)
W_n = np.power(10, W_n_dB / 10)
print("差分正极矩阵dB转化为传输率 = \n", W_p)
print("差分负极矩阵dB转化为传输率 = \n", W_n)

# 差分矩阵乘法
y_p = np.matmul(x, W_p)
y_n = np.matmul(x, W_n)

# 差分输出结果
print("差分正极输出结果x * W_p =", y_p)
print("差分负极输出结果x * W_n =", y_n)

# 差分矩阵结果相减后输出
y_diff = y_p - y_n
print("差分矩阵结果相减后输出 = ", y_diff)

# 考虑差分交叉阵列结构平均分配功率
y_out_p = y_p / 32
y_out_n = y_n / 32
print("考虑差分交叉阵列结构除以32后的实际差分正极输出光功率值y_out_p =\n", y_out_p)
print("考虑差分交叉阵列结构除以32后的实际差分负极输出光功率值y_out_n =\n", y_out_n)

# 实际差分输出功率相减后输出值
y_out_diff = y_out_p - y_out_n
print("考虑差分交叉阵列结构除以32后的实际差分输出功率相减后输出值y_out_diff =\n", y_out_diff)

# 考虑前端放大10K倍，将光功率转化为电压信号，输入到电路中进行仿真
vin = y_out_diff * 10e3
print(f"考虑前端放大10K倍，将光功率转化为电压信号=\n{vin}")

# 输出功率转化为dBm
y_dBm_p = 10 * np.log10(y_out_p / 1e-3)
y_dBm_n = 10 * np.log10(y_out_n / 1e-3)
print("差分正极转化为dBm的值 = ", y_dBm_p)
print("差分负极转化为dBm的值 = ", y_dBm_n)

# 差分输出功率dBm相减后输出值
y_diff_dBm = y_dBm_p - y_dBm_n
print("差分输出功率dBm相减后输出值 = ", y_diff_dBm)

-----神经网络第1层量化后权重部署-----
输入向量功率 = 
 [6.7 3.  5.2 2.3]
原始权重矩阵数据 = 
 [[ 2.  3. -3. -8.]
 [ 2.  3. -3. -3.]
 [ 1. -0.  2. -3.]
 [ 2.  0.  1.  2.]]
将pytorch权重矩阵转置 = 
 [[ 2.  2.  1.  2.]
 [ 3.  3. -0.  0.]
 [-3. -3.  2.  1.]
 [-8. -3. -3.  2.]]
原始矩阵乘法输出结果 =  [-11.6  -0.1  10.2  23.2]
差分正极dB矩阵 = 
 [[ -1.    -1.    -1.    -1.  ]
 [ -1.    -1.    -1.    -1.  ]
 [ -3.39  -3.39  -1.    -1.  ]
 [-20.    -3.39  -3.39  -1.  ]]
差分负极dB矩阵 = 
 [[-2.44 -2.44 -1.66 -2.44]
 [-3.39 -3.39 -1.   -1.  ]
 [-1.   -1.   -2.44 -1.66]
 [-1.   -1.   -1.   -2.44]]
差分正极矩阵dB转化为传输率 = 
 [[0.79432823 0.79432823 0.79432823 0.79432823]
 [0.79432823 0.79432823 0.79432823 0.79432823]
 [0.45814189 0.45814189 0.79432823 0.79432823]
 [0.01       0.45814189 0.45814189 0.79432823]]
差分负极矩阵dB转化为传输率 = 
 [[0.57016427 0.57016427 0.68233869 0.57016427]
 [0.45814189 0.45814189 0.79432823 0.79432823]
 [0.79432823 0.79432823 0.57016427 0.68233869]
 [0.79432823 0.79432823 0.79432823 0.57016427]]
差分正极输出结果x * W_p = [10.11032169 11.14104803 1

## SNN第2层输入脉冲信号
## **注意：第2层的输入脉冲信号不能直接从python的仿真环境中取，应该使用第1层电路仿真结果的输出脉冲！！**

In [13]:
# 输入向量（每一行代表一个时间步长，共30个时间步长）
# 鸢尾花数据集30个测试集数据
index_0_layer_2 = np.array(
             [[1e-3, 1e-3, 0e-3, 0e-3], # 第0个时间步长,0-50ns
              [0e-3, 0e-3, 0e-3, 1e-3], # 第1个时间步长,50-100ns
              [0e-3, 0e-3, 1e-3, 1e-3], # 第2个时间步长,100-150ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第3个时间步长,150-200ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第4个时间步长,200-250ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第5个时间步长,250-300ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第6个时间步长,300-350ns
              [1e-3, 1e-3, 0e-3, 1e-3], # 第7个时间步长,350-400ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第8个时间步长,400-450ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第9个时间步长,450-500ns
              [1e-3, 1e-3, 1e-3, 1e-3], # 第10个时间步长,500-550ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第11个时间步长,550-600ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第12个时间步长,600-650ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第13个时间步长,650-700ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第14个时间步长,700-750ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第15个时间步长,750-800ns
              [0e-3, 0e-3, 1e-3, 1e-3], # 第16个时间步长,800-850ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第17个时间步长,850-900ns
              [0e-3, 0e-3, 0e-3, 1e-3], # 第18个时间步长,900-950ns 
              [0e-3, 1e-3, 0e-3, 0e-3], # 第19个时间步长,950-1000ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第20个时间步长,1000-1050ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第21个时间步长,1050-1100ns
              [1e-3, 1e-3, 0e-3, 0e-3], # 第22个时间步长,1100-1150ns
              [1e-3, 1e-3, 0e-3, 1e-3], # 第23个时间步长,1150-1200ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第24个时间步长,1200-1250ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第25个时间步长,1250-1300ns
              [1e-3, 1e-3, 0e-3, 1e-3], # 第26个时间步长,1300-1350ns
              [0e-3, 0e-3, 0e-3, 0e-3], # 第27个时间步长,1350-1400ns
              [1e-3, 1e-3, 0e-3, 1e-3], # 第28个时间步长,1400-1450ns
              [0e-3, 0e-3, 0e-3, 0e-3]] # 第29个时间步长,1450-1500ns
            )

index_10_layer_2 = np.array(
             [[0e-3, 0e-3, 0e-3, 0e-3], # 第0个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第1个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第2个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第3个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第4个时间步长
              [1e-3, 1e-3, 0e-3, 0e-3], # 第5个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第6个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第7个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第8个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第9个时间步长
              [1e-3, 1e-3, 0e-3, 0e-3], # 第10个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第11个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第12个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第13个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第14个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第15个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第16个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第17个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第18个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第19个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第20个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第21个时间步长
              [1e-3, 1e-3, 0e-3, 0e-3], # 第22个时间步长
              [1e-3, 1e-3, 1e-3, 0e-3], # 第23个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第24个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第25个时间步长
              [1e-3, 1e-3, 1e-3, 1e-3], # 第26个时间步长
              [0e-3, 0e-3, 0e-3, 0e-3], # 第27个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第28个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3]] # 第29个时间步长
              )

index_20_layer_2 = np.array(
             [[0e-3, 0e-3, 0e-3, 1e-3], # 第0个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第1个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第2个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第3个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第4个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第5个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第6个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第7个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第8个时间步长
              [0e-3, 0e-3, 1e-3, 1e-3], # 第9个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第10个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第11个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第12个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第13个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第14个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第15个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第16个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第17个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第18个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第19个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第20个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第21个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第22个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第23个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第24个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第25个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第26个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第27个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3], # 第28个时间步长
              [0e-3, 0e-3, 0e-3, 1e-3]] # 第29个时间步长
              )

## 神经网络第2层量化后权重部署

In [20]:
print("-----神经网络第2层量化后权重部署-----")

# 输入向量（从上一个代码块中选择）
x = index_20_layer_2
print("输入向量功率 = \n", x)

# 测试输入向量
# x = np.array([0, 0, 0.03571106, 0.08124441])
# print("输入向量功率 = \n", x)

# 原始权重矩阵数据
W = np.array([[ 7.,  4., -1., -2.],
              [-6.,  2.,  3.,  1.],
              [-8., -6., -2.,  3.]])
print("原始权重矩阵数据 = \n", W)

# 将pytorch权重矩阵转置
W = W.T
print("将pytorch权重矩阵转置 = \n", W)

# 将权重映射到dB
W_p_dB, W_n_dB = weight_to_dB(W, mapping_dict_pos, mapping_dict_neg)

# 差分正极dB矩阵
print("差分正极dB矩阵 = \n", W_p_dB)

# 差分负极dB矩阵
print("差分负极dB矩阵 = \n", W_n_dB)

# 差分正极传输率矩阵
# W_p = np.array([[0.794, 0.794, 0.199, 0.01],
#                 [0.794, 0.794, 0.199, 0.199],
#                 [0.794, 0.794, 0.794, 0.199],
#                 [0.794, 0.794, 0.794, 0.794]])

# 差分负极传输率矩阵
# W_n = np.array([[0.316, 0.199, 0.794, 0.794],
#                 [0.316, 0.199, 0.794, 0.794],
#                 [0.501, 0.794, 0.316, 0.794],
#                 [0.316, 0.794, 0.501, 0.316]])

# 传输率转化为dB
# W_p_dB = 10 * np.log10(W_p / 1)
# W_n_dB = 10 * np.log10(W_n / 1)
# print("差分正极矩阵传输率转化为dB = \n", W_p_dB)
# print("差分负极矩阵传输率转化为dB = \n", W_n_dB)

# dB转化为传输率
W_p = np.power(10, W_p_dB / 10)
W_n = np.power(10, W_n_dB / 10)
print("差分正极矩阵dB转化为传输率 = \n", W_p)
print("差分负极矩阵dB转化为传输率 = \n", W_n)

# 差分矩阵乘法
y_p = np.matmul(x, W_p)
y_n = np.matmul(x, W_n)

# 差分输出结果
print("差分正极输出结果x * W_p =", y_p)
print("差分负极输出结果x * W_n =", y_n)

# 差分矩阵结果相减后输出
y_diff = y_p - y_n
print("差分矩阵结果相减后输出 = ", y_diff)

# 考虑差分交叉阵列结构平均分配功率
y_out_p = y_p / 32
y_out_n = y_n / 32
print("考虑差分交叉阵列结构除以32后的实际差分正极输出光功率值y_out_p =\n", y_out_p)
print("考虑差分交叉阵列结构除以32后的实际差分负极输出光功率值y_out_n =\n", y_out_n)

# 实际差分输出功率相减后输出值
y_out_diff = y_out_p - y_out_n
print("考虑差分交叉阵列结构除以32后的实际差分输出功率相减后输出值y_out_diff =\n", y_out_diff)

# 考虑前端放大10K倍，将光功率转化为电压信号，输入到电路中进行仿真
vin = y_out_diff * 10e3
print(f"考虑前端放大10K倍，将光功率转化为电压信号=\n{vin}")

# 输出功率转化为dBm
y_dBm_p = 10 * np.log10(y_out_p / 1e-3)
y_dBm_n = 10 * np.log10(y_out_n / 1e-3)
print("差分正极转化为dBm的值 = ", y_dBm_p)
print("差分负极转化为dBm的值 = ", y_dBm_n)

# 差分输出功率dBm相减后输出值
y_diff_dBm = y_dBm_p - y_dBm_n
print("差分输出功率dBm相减后输出值 = ", y_diff_dBm)

-----神经网络第2层量化后权重部署-----
输入向量功率 = 
 [[0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.001 0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]
 [0.    0.    0.    0.001]]
原始权重矩阵数据 = 
 [[ 7.  4. -1. -2.]
 [-6.  2.  3.  1.]
 [-8. -6. -2.  3.]]
将pytorch权重矩阵转置 = 
 [[ 7. -6. -8.]
 [ 4.  2. -6.]
 [-1.  3. -2.]
 [-2.  1.  3.]]
差分